<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-11 11:49:40
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.32 C
-------------------
Today PnL: -46.29 K (-0.35%)
Current PnL: -23.04 L (-15.67%)
CY Booked + Current PnL: -14.77 L (-10.05%)
-------------------
Total profit:  2.01 L
Total loss:  -25.05 L
-------------------
Total Booked + Current PnL: 12.21 L (10.17%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.27%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 82.52 L (62.59%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 6.03%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-12.30,M-LC,4.00,217658.0,16178.0,11035.0,2.12,8.03,5.07,13.51,61.0,1.47,1.68,21.64,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,12.80,H-MC,13.90,161159.0,34959.0,11217.0,-2.38,27.70,6.96,36.59,111.0,3.12,1.25,43.69,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-22.38,H-LC,5.70,107304.0,2992.0,13660.0,-0.31,2.87,12.73,15.96,11.0,0.22,0.83,20.25,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,93.81,M-SC,10.94,83545.0,-17232.0,17327.0,0.62,-17.10,20.74,0.09,243.0,-0.99,0.65,9.76,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-26.68,H-SC,8.66,218960.0,25940.0,22093.0,0.93,13.44,10.09,24.89,134.0,1.17,1.69,39.92,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ICICIPRULI,600.83,790.00,-20.77,H-MC,2.12,136357.0,569.0,42189.0,-1.10,0.42,30.94,31.48,109.0,0.01,1.05,12.59,X40,ATH,INSURANCE
25,DABUR,505.20,735.00,-9.60,H-MC,4.25,196892.0,-136.0,89763.0,-0.47,-0.07,45.59,45.49,103.0,-0.00,1.52,12.96,XY24,BTT,FMCG
58,PGHH,13388.00,18062.58,-31.21,H-MC,6.55,199125.0,-1695.0,71804.0,0.39,-0.84,36.06,34.92,82.0,-0.02,1.54,3.79,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.87,H-LC,1.98,205610.0,1110.0,40300.0,1.01,0.54,19.60,20.25,10.0,0.03,1.59,9.32,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.45,H-LC,2.41,168871.0,-32515.0,120135.0,0.31,-16.15,71.14,43.51,27.0,-0.27,1.31,17.55,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-9.25,H-LC,3.60,180324.0,-19986.0,79956.0,-0.06,-9.98,44.34,29.94,17.0,-0.25,1.39,24.58,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-16.50,H-LC,3.94,128536.0,-136890.0,193794.0,0.19,-51.57,150.77,21.44,8.0,-0.71,0.99,3.95,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.0,-21.72,H-LC,5.42,156420.0,243.0,25856.0,0.36,0.16,16.53,16.71,19.0,0.01,1.21,17.47,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,12.48,H-SC,13.18,133925.0,8628.0,50369.0,4.98,6.89,37.61,47.09,141.0,0.17,1.04,67.62,MH,ATH,POWER
71,SIEMENS,4406.90,7969.85,-3.47,H-LC,36.11,77635.0,-32537.0,121607.0,2.43,-29.53,156.64,80.85,23.0,-0.27,0.60,14.34,AR,ATH,ELECTRICAL
79,TATAMOTORS,761.77,1065.00,-25.46,M-LC,5.54,235587.0,-40936.0,151011.0,2.41,-14.80,64.10,39.81,55.0,-0.27,1.82,12.91,XY24,NTT,AUTO
68,SBIN,760.30,863.00,-12.30,M-LC,4.00,217658.0,16178.0,11035.0,2.12,8.03,5.07,13.51,61.0,1.47,1.68,21.64,XY25,NTT,BANKS
20,CERA,7989.07,9475.00,-20.29,H-SC,3.06,113823.0,-29980.0,56729.0,1.90,-20.85,49.84,18.60,148.0,-0.53,0.88,25.61,OX40N,NTT,CERAMICS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VOLTAS,1278.28,1918.49,-12.41,H-MC,8.12,183675.0,-8067.0,104107.0,-6.16,-4.21,56.68,50.08,101.0,-0.08,1.42,2.30,XY25,ATH,AC
43,INDIGOPNTS,1407.73,1408.00,104.80,M-SC,3.71,134118.0,-40441.0,40477.0,-6.14,-23.17,30.18,0.02,221.0,-1.00,1.04,16.30,OX40N,NTT,PAINTS
29,EASEMYTRIP,18.12,26.40,-11.01,M-SC,22.96,85675.0,-91140.0,171933.0,-3.62,-51.55,200.68,45.70,197.0,-0.53,0.66,0.00,XY24,NTT,TRAVEL
84,UJJIVANSFB,52.77,60.00,82.03,H-SC,20.58,114939.0,-27540.0,47056.0,-3.47,-19.33,40.94,13.70,163.0,-0.59,0.89,35.66,OX40N,NTT,BANKS
52,LAOPALA,369.40,464.00,105.71,H-SC,3.15,65438.0,-35408.0,61237.0,-3.11,-35.11,93.58,25.61,144.0,-0.58,0.51,21.32,AR,NTT,CERAMICS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.81,M-SC,10.94,83545.0,-17232.0,17327.0,0.62,-17.10,20.74,0.09,243.0,-0.99,0.65,9.76,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,82.03,H-SC,20.58,114939.0,-27540.0,47056.0,-3.47,-19.33,40.94,13.70,163.0,-0.59,0.89,35.66,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-20.29,H-SC,3.06,113823.0,-29980.0,56729.0,1.90,-20.85,49.84,18.60,148.0,-0.53,0.88,25.61,OX40N,NTT,CERAMICS
72,SIS,477.00,528.0,2245.60,H-SC,7.07,63104.0,-17986.0,26655.0,-0.42,-22.18,42.24,10.69,155.0,-0.67,0.49,27.89,OX40N,NTT,MISC
78,TATAELXSI,7332.28,9161.0,-20.18,H-MC,2.35,79611.0,-23041.0,48642.0,-1.27,-22.45,61.10,24.94,99.0,-0.47,0.62,21.02,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.52,237128.0,-58806.0,152710.0,0.12,-19.87,64.40,31.73,2.0,-0.39,1.83,1.24,X40,BTT,IT
45,INFY,1461.46,2275.00,-22.11,H-LC,9.83,254932.0,-6669.0,152296.0,0.08,-2.55,59.74,55.67,4.0,-0.04,1.97,3.40,X40,BTT,IT
89,VBL,492.64,671.64,-10.71,H-LC,10.94,271260.0,6220.0,90085.0,1.66,2.35,33.21,36.33,5.0,0.07,2.10,15.96,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.71,H-LC,9.71,197866.0,-18653.0,111557.0,1.03,-8.61,56.38,42.91,7.0,-0.17,1.53,3.62,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.50,H-LC,3.94,128536.0,-136890.0,193794.0,0.19,-51.57,150.77,21.44,8.0,-0.71,0.99,3.95,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.8,H-MC,13.9,161159.0,34959.0,11217.0,-2.38,27.7,6.96,36.59,111.0,3.12,1.25,43.69,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.80,H-MC,13.90,161159.0,34959.0,11217.0,-2.38,27.70,6.96,36.59,111.0,3.12,1.25,43.69,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,12.48,H-SC,13.18,133925.0,8628.0,50369.0,4.98,6.89,37.61,47.09,141.0,0.17,1.04,67.62,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,2.13,H-MC,7.66,176330.0,11660.0,56302.0,-2.06,7.08,31.93,41.28,91.0,0.21,1.36,15.63,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-8.49,H-SC,2.70,103964.0,1960.0,41970.0,-0.85,1.92,40.37,43.06,123.0,0.05,0.80,19.37,X40N,ATH,MISC
68,SBIN,760.30,863.00,-12.30,M-LC,4.00,217658.0,16178.0,11035.0,2.12,8.03,5.07,13.51,61.0,1.47,1.68,21.64,XY25,NTT,BANKS


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,12.48,H-SC,13.18,133925.0,8628.0,50369.0,4.98,6.89,37.61,47.09,141.0,0.17,1.04,67.62,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.53,159218.0,8305.0,48991.0,-1.10,5.50,30.77,37.97,92.0,0.17,1.23,32.20,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,12.86,134784.0,11448.0,122316.0,-0.71,9.28,90.75,108.45,119.0,0.09,1.04,33.71,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-44.07,M-SC,4.47,94508.0,3010.0,75739.0,-0.36,3.29,80.14,86.07,219.0,0.04,0.73,36.71,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-15.61,M-LC,6.52,148211.0,-2734.0,112196.0,-1.35,-1.81,75.70,72.51,54.0,-0.02,1.15,4.10,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.26,69082.0,-44467.0,84522.0,-0.19,-39.16,122.35,35.27,268.0,-0.53,0.53,69.08,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-5.01,H-SC,17.36,88391.0,-12572.0,107103.0,-0.41,-12.45,121.17,93.63,149.0,-0.12,0.68,31.66,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.96,H-SC,15.86,90060.0,-7920.0,29792.0,-0.73,-8.08,33.08,22.32,152.0,-0.27,0.70,30.62,XR,ATH,FINANCE
84,UJJIVANSFB,52.77,60.00,82.03,H-SC,20.58,114939.0,-27540.0,47056.0,-3.47,-19.33,40.94,13.70,163.0,-0.59,0.89,35.66,OX40N,NTT,BANKS
50,KPIGREEN,497.21,731.35,12.48,H-SC,13.18,133925.0,8628.0,50369.0,4.98,6.89,37.61,47.09,141.0,0.17,1.04,67.62,MH,ATH,POWER


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VBL,492.64,671.64,-10.71,H-LC,10.94,271260.0,6220.0,90085.0,1.66,2.35,33.21,36.33,5.0,0.07,2.10,15.96,X40N,ATH,FMCG
70,SHALBY,261.39,327.00,933.49,M-SC,7.99,140540.0,-40603.0,86067.0,-0.23,-22.42,61.24,25.10,232.0,-0.47,1.09,12.63,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-26.68,H-SC,8.66,218960.0,25940.0,22093.0,0.93,13.44,10.09,24.89,134.0,1.17,1.69,39.92,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.53,159218.0,8305.0,48991.0,-1.10,5.50,30.77,37.97,92.0,0.17,1.23,32.20,AR,ATH,AUTO
55,MEDANTA,1087.93,1486.00,12.80,H-MC,13.90,161159.0,34959.0,11217.0,-2.38,27.70,6.96,36.59,111.0,3.12,1.25,43.69,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.49
1,25,41.13
2,50,70.87


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.51
LC    34.10
MC    26.39
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.99
X40N     15.35
X40      13.43
XY25     10.70
XR        8.97
AR        8.44
OX40N     7.58
X5K       2.23
MH        1.80
X200      1.39
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.80
H-SC    24.60
H-MC    23.44
M-SC    13.46
M-LC     7.22
M-MC     2.61
L-SC     1.45
L-LC     1.08
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.34,-11.11,46.82
IT,12.03,-19.99,79.92
FINANCE,8.33,-21.19,66.03
BANKS,7.64,-15.37,64.35
MISC,5.91,-20.13,88.99
PAINTS,5.67,-19.25,42.75
HEALTHCARE,5.24,3.03,21.45
ELECTRICAL,4.73,-14.84,64.57
AUTO,4.43,-17.89,69.18


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2890362.0,23
XR,1133637.0,13
AR,1117124.0,10
X40,834183.0,10
XY25,663216.0,8
X40N,621981.0,12
OX40N,480097.0,10
SR,197790.0,2
X5K,150535.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2836691.0,28
M-SC,1505040.0,17
H-LC,1399719.0,18
H-MC,1345939.0,18
M-LC,445516.0,5
M-MC,340668.0,2
L-SC,273635.0,3
L-MC,58045.0,1
L-LC,46484.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      950428.0      6
M-SC       XY24      878817.0      7
H-SC       AR        673080.0      5
           XR        533899.0      6
H-LC       X40       522864.0      5
H-MC       XY24      489889.0      5
M-MC       XY24      340668.0      2
H-LC       XY25      254393.0      3
           X40N      250532.0      5
H-MC       X40       235666.0      4
H-LC       AR        233164.0      2
H-SC       X40N      226817.0      4
H-MC       XY25      225643.0      2
H-SC       OX40N     204308.0      4
           SR        197790.0      2
M-LC       XY24      185410.0      2
L-SC       XR        182860.0      2
M-SC       AR        161889.0      2
H-MC       X40N      144632.0      3
M-LC       X5K       136875.0      1
M-SC       OX40N     136372.0      4
           XR        126648.0      2
           XY25      125661.0      1
H-MC       XR        119989.0      1
M-LC       XR        112196.0      1
L-SC       OX40N      90775.0      1
H-LC       X200       79956.0      1
M-SC       X40        75653.0      1
L-MC       XR         58045.0      1
H-SC       MH         50369.0      1
H-MC       AR         48991.0      1
           OX40N      48642.0      1
L-LC       XY25       46484.0      1
H-LC       XY24       45150.0      1
H-MC       MH         32487.0      1
H-LC       X5K        13660.0      1
M-LC       XY25       11035.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
